# El juego de la vida de John H. Conway

Es un autómata celular desarrollado por el matemático británico John Horton Conway
 
- En inglés, _The Game of Life_ o, simplemente, Life

Es un "juego" de cero jugadores

- Vamos, que de juego tiene poco; mejor dicho, es una simulación
- Se parte de un estado inicial y se deja que el sistema evolucione

Su funcionamiento se basa en cuatro reglas muy simples:

1. Cualquier célula **viva** con menos de dos vecinas vivas **muere** (hambruna)
2. Cualquier célula **viva** con dos o tres vecinos vivos **vive** en la siguiente generación
3. Cualquier célula **viva** con más de tres vecinos vivos **muere**, (exceso de población)
4. Cualquier célula **muerta** con exactamente tres vecinas vivas se convierte en una célula **viva**, (reproducción)

Primero, vamos a importar las librerías que usaremos a lo largo de la presentación

In [1]:
import pprint as pp
from random import randrange
from time import sleep

from IPython.display import clear_output

from life import Universe, board, shift, pattern

## Implementación

Vamos a realizar una pequeña implementación del autómata

Así tendremos algo con lo que ilustrar ejemplos y patrones

### Representando el escenario de juego

Vamos a crear una clase `Universe`, que representará el escenario donde evolucionan las células

```python
class Universe:
```

Las células se representarán como un conjunto de posiciones $(x, y)$, 
  - Por ejemplo, `{(1, 1), (1, 2), (1, 3), (2, 1)}`
  - De esta manera nos vale tanto para cualquier dimensión de mapa

El universo evolucionará con el tiempo según las reglas definidas

- Las reglas se aplican simultáneamente en todas las células
- El resultado tras aplicarlas será una nueva **generación** de dicho universo

```python
class Universe:
    def __init__(self, cells=None):
        self.cells = set(cells) if cells else {}
        self.generation = 0
```

Para avanzar una generación nos apoyaremos en dos métodos auxiliares:

1. `neighbours`, que determinará las posiciones de los vecinos de una célula concreta

```python
    ...
    def neighbours(self, cell):
        for x, y in product(range(-1, 2), repeat=2):
            if x != and y != 0:
                yield cell[0] + x, cell[1] + y
```

2. `neighbours_activation`, que dterminará cuántas veces aparece cada uno de los vecinos activados

```python
    ...
    def active_neighbours(self):
        """The neighbours alongside the times they're are activated."""
        return Counter(neighbour for cell in self.cells for neighbour in self.neighbours(cell=cell))
```

Veamos cómo funcionan

In [2]:
universe = Universe({(0, 0), (1, 0)})
print(f'Cells in universe at generation {universe.generation}: {universe.cells}')

Cells in universe at generation 0: {(1, 0), (0, 0)}


In [3]:
cell = (0, 0)
print(f'Neighbours for cell {cell}: {list(universe.neighbours(cell=cell))}')

Neighbours for cell (0, 0): [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]


In [4]:
print(f'Activations: {pp.pformat(universe.active_neighbours())}')

Activations: Counter({(0, -1): 2,
         (0, 1): 2,
         (1, -1): 2,
         (1, 1): 2,
         (0, 0): 1,
         (2, -1): 1,
         (2, 0): 1,
         (2, 1): 1,
         (-1, -1): 1,
         (-1, 0): 1,
         (-1, 1): 1,
         (1, 0): 1})


Ya tenemos todo lo necesario para hacer avanzar una generación nuestro universo. Lo implementaremos en el método `step`:

- Usará el método `neighbours_count` para ver qué células se activarán y por cuántos vecinos vivos
- Filtrará aquellas que, o bien tienen exactamente 3 vecinos, o bien 2 pero que ya estaban vivas antes
- Por último, hará avanzar en 1 las generaciones

```python
    ...
    def step(self):
        self.cells = {
            c for c, n in self.active_neighbours().items()
            if n == 3 or (n == 2 and c in self.cells)
        }
        self.generation += 1
```

In [5]:
universe = Universe({(-1, 0), (0, 1), (1, 0), (0, 0)})
for _ in range(5):
    print(f'Generation {universe.generation}: {list(universe.cells)}')
    universe.step()

Generation 0: [(-1, 0), (1, 0), (0, 1), (0, 0)]
Generation 1: [(0, 1), (0, 0), (-1, 1), (1, 1), (-1, 0), (1, 0), (0, -1)]
Generation 2: [(-1, -1), (-1, 1), (1, 1), (1, -1), (0, 2), (0, -1)]
Generation 3: [(0, 1), (0, -1), (-1, 0), (0, 2), (1, 0), (0, -2)]
Generation 4: [(0, 1), (-1, -1), (-1, 1), (1, 1), (1, -1), (-1, 0), (1, 0), (0, -1)]


¡Ya tenemos nuestro simulador funcionando! Eso sí, tenemos que hacer que sea más cómodo de visualizar

Antes de terminar vamos a añadir una propiedad, `size` que determinará cuántas células vivas quedan

```python
    ...
    @property
    def size(self):
        return len(self.cells)
```

- Nos puede servir por ejemplo, para no hacer evolucionar más un universo que esté vacío

La implementación completa (con comentarios) se encuentra en el módulo `life.py`

## Visualización

Vamos a implementar la visualización en modo texto.

- De esto se encargará la función `board`, que devolverá el tablero como cadena de texto
- Admitirá como parámetros el universo a pinter y el tamaño del tablero
- Opcionalmente, podremos usar diferentes estilos para las células vivas y muertas

```python
def board(universe, size, ok='⬛', ko='⬜'):
    return '\n'.join(
        ''.join(ok if (x, y) in universe.cells else ko for x in range(width))
        for y in range(height)
    )
```

Ahora vamos a darle una vuelta de tuerca. Vamos a convertir el método `board` en un generador

- Le añadiremos un parámetro opcional `generations` que dirá cuántas generaciones queremos producir
- Y como es opcional, si no se especifica nada, ¡que no pare!

```python
def board(universe, size, gens=None, ok='⬛', ko='⬜'):
    width, height = size
    while generations is None or universe.generation < gens:
        yield '\n'.join(
        ''.join(
            ok if (x, y) in universe.cells else ko for x in range(width))
            for y in range(height)
        )
        universe.step()
```

Veamos cómo funciona:

In [6]:
universe = Universe({(1, 1), (2, 3), (2, 1), (3, 1), (6, 6)})
board_generator = board(universe, size=(5, 5), gens=5)
print(f'Starting universe (generation {universe.generation}):\n{next(board_generator)}')
for b in board_generator:
    print(f'Universe in generation {universe.generation}:\n{b}')

Starting universe (generation 0):
⬜⬜⬜⬜⬜
⬜⬛⬛⬛⬜
⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜
Universe in generation 1:
⬜⬜⬛⬜⬜
⬜⬜⬛⬜⬜
⬜⬛⬜⬛⬜
⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜
Universe in generation 2:
⬜⬜⬜⬜⬜
⬜⬛⬛⬛⬜
⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜
Universe in generation 3:
⬜⬜⬛⬜⬜
⬜⬛⬛⬛⬜
⬜⬛⬛⬛⬜
⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜
Universe in generation 4:
⬜⬛⬛⬛⬜
⬜⬜⬜⬜⬜
⬜⬛⬜⬛⬜
⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜


Y ahora, como estamos usando jupyter, vamos a aprovechar sus capacidades para la animación

Básicamente imprimiremos los tableros de cada generación en una determinada frecuencia y limpiaremos la salida entre dibujos

In [7]:
width, height = 10, 10
freq = 15
generations = 250

universe = Universe({(randrange(width), randrange(height)) for _ in range(width*height)})

for b in board(universe, size=(width, height), gens=generations):
    print(f'Generation: {universe.generation}; size: {universe.size}\n{b}')
    clear_output(wait=True)
    sleep(1 / freq)

Generation: 249; size: 43
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬛⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜


Encapsularemos esta funcionalidad en una función, para no repetirnos demasiado:

In [8]:
def run(universe, size, gens, freq):
    for b in board(universe, size=size, gens=gens):
        print(f'Generation: {universe.generation}; size: {universe.size}\n{b}')
        clear_output(wait=True)
        sleep(1 / freq)

## Patrones

Ahora vamos a conocer algunos de los patrones más comunes que existen

- Patrones: Configuraciones con ciertas propiedades

Eso sí, como sería una locura escribir en base a cordenadas $(x, y)$ todas las figuras, nos apoyaremos en un par de funciones:

- `pattern`, que devolverá el conjunto de células a partir de un patrón de texto

```python
def pattern(shape, live='#'):
    return {
        (x, y) 
        for (y, row) in enumerate(shape.strip().splitlines())
        for (x, c) in enumerate(row)
        if c == live
    }
```

- `shift`, que desplazará el patrón hacia una posición del tablero (el $(0, 0)$ es la posición superior izquierda)

```python
def shift(cells, dx, dy):
    return {(x + dx, y + dy) for (x, y) in cells}
```

Esta implementación se basa en la propuesta por [Peter Norvig](https://github.com/norvig/pytudes/blob/main/ipynb/Life.ipynb) para su implementación del Juego de la Vida

Veamos su funcionamiento con algunos patrones:

- `blinker`, ejemplo de _oscilador_ (_oscillator_) de periodo dos
- `toad`, otro ejemplo de _oscilador_ de periodo dos
- `beehive`, ejemplo de _naturaleza muerta_ (_still_life_)

In [9]:
blinker = pattern("""###""")
toad    = pattern("""
.###
###.
""")
beehive = pattern("""
.##.
#..#
.##.
""")

cells = shift(blinker, 1, 2) | shift(toad, 11, 2) | shift(beehive, 21, 1)
universe=Universe(cells)

run(universe, size=(26, 6), gens=20, freq=5)

Generation: 19; size: 15
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜⬜⬛⬛⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜⬜⬛⬜⬜⬛⬜⬜⬜⬜⬜⬜⬛⬜⬜⬛⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜⬜⬛⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜⬛⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜


Como os podréis imaginar, existen miles de patrones, clasificados en decenas de categorías diferentes- Pero nosotros pararemos aquí

- Si os habéis quedado con ganas, la [Life Wiki](https://www.conwaylife.com/wiki/Main_Page) es un repositorio de información gigantesco, con información tanto del Juego de la vida como de sus variantes

Ahora, vamos a jugar con algoritmos genéticos en este problema